# A\*, IDS, and Effective Branching Factor

## Heuristic Functions

For `aStarSearch` use the following two heuristic functions, plus one more of your own design, for a total of three heuristic functions.

  * `h1_8p(state, goal)`: $h(state, goal) = 0$, for all states $state$ and all goal states $goal$,
  * `h2_8p(state, goal)`: $h(state, goal) = m$, where $m$ is the Manhattan distance that the blank is from its goal position,
  * `h3_8p(state, goal)`: $h(state, goal) = ?$, that you define.  It must be admissible, and not constant for all states.

## Comparison

Apply all four algorithms (`iterativeDeepeningSearch` plus `aStarSearch` with the three heuristic
functions) to three eight-tile puzzle problems with start state

$$
\begin{array}{ccc}
1 & 2 & 3\\
4 & 0 & 5\\
6 & 7 & 8
\end{array}
$$

and these three goal states.

$$
\begin{array}{ccccccccccc}
1 & 2 & 3  & ~~~~ & 1 & 2 & 3  &  ~~~~ & 1 & 0 &  3\\
4 & 0 & 5  & & 4 & 5 & 8  & & 4 & 5 & 8\\
6 & 7 & 8 &  & 6 & 0 & 7  & & 2 & 6 & 7
\end{array}
$$

Print a well-formatted table like the following.  Try to match this
format. 

           [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
    Algorithm    Depth  Nodes  EBF              Depth  Nodes  EBF              Depth  Nodes  EBF          
         IDS       0      0  0.000                3     43  3.086               11 225850  2.954         
        A*h1       0      0  0.000                3    116  4.488               11 643246  3.263         
        A*h2       0      0  0.000                3     51  3.297               11 100046  2.733         

Of course you will have one more line for `h3`.

Insert your functions here.

In [1]:
# from A3mysolution import *

First, some example output for the ebf function.

In [2]:
import numpy as np
import math
import copy
import pandas as pd

In [3]:
nodes=0
netdepth=0

def ebf(nNodes, depth, precision=0.01):
    if depth==0:
        if nNodes==1:
            return 1.0
        else:
            return 0.0
    global nodes
    nodes=nNodes
    global netdepth
    netdepth=depth
    precision=precision
    x0=nNodes**(1/(depth+1))
    result = solve(quadratic, x0, precision) 
    return result

def derivative(f, x, h):
      return (f(x+h) - f(x-h)) / (2.0*h)  

def quadratic(x):
    return ((1-x**(netdepth +1))/(1-x))-nodes     

def solve(f, x0, h):
    lastX = x0
    nextX = lastX + 10* h  
    while (abs(lastX - nextX) > h):  
        newY = f(nextX)                        
        lastX = nextX
        nextX = lastX - newY / derivative(f, lastX, h)  
    return nextX        

In [4]:
ebf(10, 3)

1.6608024572821003

The smallest argument values should be a depth of 0, and 1 node.

In [5]:
ebf(1, 0)

1.0

In [6]:
ebf(0,0)

0.0

In [7]:
ebf(2, 1)

1.0000000000000007

In [8]:
ebf(2, 1, precision=0.000001)

0.9999999999659235

In [9]:
ebf(200000, 5)

11.275597027190864

In [10]:
ebf(200000, 50)

1.2374964630891387

Here is a simple example using our usual simple graph search.

In [11]:
def findBlank_8p(s):
    return iTorc(s.index(0))

def actionsF_8p(state):
    r, c = findBlank_8p(state)
    actions = []
    if c > 0:
        actions.append(('left',1))
    if c < 2:
        actions.append(('right',1))
    if r > 0:
        actions.append(('up',1))
    if r < 2:
        actions.append(('down',1))
    return actions


def takeActionF_8p(state, action):
    import copy
    state = copy.deepcopy(state)
    r, c = findBlank_8p(state)
    dr = dc = 0
    if action == (('left',1)):
        dc -= 1
    elif action == (('right',1)):
        dc += 1
    elif action == (('up',1)):
        dr -= 1
    elif action == (('down',1)):
        dr += 1
    newr, newc = r+dr, c+dc
    setTile(state, r, c, getTile(state, newr, newc))
    setTile(state, newr, newc, 0)
    return (state,1)

def setTile(state, row, col, tile):
    state[rcToi(row, col)] = tile
    return state


def getTile(state, row, col):
    return state[rcToi(row, col)]

def rcToi(row, col):
    return row*3+col


def iTorc(i):
    row = i // 3
    col = i - row*3
    return (row, col)

In [12]:
actions=actionsF_8p([1,2,3,0,5,6,7,8])
actions

[('right', 1), ('up', 1), ('down', 1)]

In [13]:
takeActionF_8p([1,2,3,0,4,5,6,7,8], ('left',1))

([1, 2, 0, 3, 4, 5, 6, 7, 8], 1)

In [14]:
def depthLimitedSearch(state, goalState, actionsF, takeActionF, depthLimit):
    if state == goalState:
        return []
    if depthLimit == 0:
        return 'cutoff'
    cutoffOccurred = False
    for action in actionsF(state):
        global COUNT
        COUNT+=1
        childState,_ = takeActionF(state, action)
        result = depthLimitedSearch(childState, goalState, actionsF, takeActionF, depthLimit-1)
        if result == 'cutoff':
            cutoffOccurred = True      
        elif result != 'failure':
            result=[childState]+result
            return result
    if cutoffOccurred == True:
        return 'cutoff'
    else:
        return 'failure'

In [15]:
COUNT = 0

def incCount():
    global COUNT
    COUNT += 1
    
def getNodeCount():
    global COUNT
    tmp = COUNT
    COUNT = 0
    return tmp

def iterativeDeepeningSearch(startState, goalState, actionsF, takeActionF, maxDepth):
    global COUNT
    COUNT=0
    for depth in range(maxDepth):
        result = depthLimitedSearch(startState, goalState, actionsF, takeActionF, depth)
        if result == 'failure':
            return 'failure'
        if result != 'cutoff':
            result=[startState]+result 
            return result
    print (COUNT)
    return 'cutoff'

In [16]:
iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], [0, 2, 3, 1, 4,  6, 7, 5, 8],actionsF_8p, takeActionF_8p, 10)

28739


'cutoff'

In [17]:
def actionsF_simple(state):
    succs = {'a': ['b', 'c'], 'b':['e', 'f', 'g'], 'b':['a'], 'c':['h'], 'h':['i'], 'i':['j', 'k', 'l'], 'k':['z']}
    return [(s, 1) for s in succs.get(state, [])]

def takeActionF_simple(state, action):
    return action

def goalTestF_simple(state, goal):
    return state == goal

def h_simple(state, goal):
    return 1

In [60]:
goalTestF_simple([5, 2, 8, 0, 1, 4, 3, 7, 6], [0, 2, 3, 1, 4,  6, 7, 5, 8])

False

In [18]:
def goalTestF_8p(state, goal):
    return goal

In [19]:
def h1_8p(state, goal):
    return 0

In [20]:
def h2_8p(state, goal):
    x=findblank_8p(state)
    y=findblank_8p(goal)
    return abs(y[0]-x[0]) + abs(y[1]-x[1])

In [21]:
def h3_8p(state, goal):
    x = 0
    for i in range(len(state)):
        if state[i] != goal[i]:
            x = x + 1          
    return x

In [22]:
actions = actionsF_simple('a')
actions

[('b', 1), ('c', 1)]

In [23]:
takeActionF_simple('a', actions[0])

('b', 1)

In [24]:
goalTestF_simple('a', 'a')

True

In [25]:
h_simple('a', 'z')

1

In [26]:
iterativeDeepeningSearch('a', 'z', actionsF_simple, takeActionF_simple, 10)

['a', 'c', 'h', 'i', 'k', 'z']

In [58]:
NCOUNT = 0

def getNCount():
    global NCOUNT
    tmp = NCOUNT
    NCOUNT = 0
    return tmp

class Node:
    def __init__(self, state, f=0, g=0 ,h=0):
        self.state = state
        self.f = f
        self.g = g
        self.h = h
    def __repr__(self):
        return "Node(" + repr(self.state) + ", f=" + repr(self.f) + \
               ", g=" + repr(self.g) + ", h=" + repr(self.h) + ")"

def aStarSearch(startState, actionsF, takeActionF, goalTestF, hF):
    h = hF(startState)
    startNode = Node(state=startState, f=0+h, g=0, h=h)
    return aStarSearchHelper(startNode, actionsF, takeActionF, goalTestF, hF, float('inf'))

def aStarSearchHelper(parentNode, actionsF, takeActionF, goalTestF, hF, fmax):
    global NCOUNT
    if goalTestF(parentNode.state):
        #print(goalTestF(parentNode.state))
        print(1)
        return ([parentNode.state], parentNode.g)
    ## Construct list of children nodes with f, g, and h values
    actions = actionsF(parentNode.state)
    if not actions:
        #print(1)
        return ("failure", float('inf'))
    children = []
    for action in actions:
        (childState,stepCost) = takeActionF(parentNode.state, action)
        h = hF(childState)
        NCOUNT += 1
        g = parentNode.g + stepCost
        f = max(h+g, parentNode.f)
        childNode = Node(state=childState, f=f, g=g, h=h)
        children.append(childNode)
        #print(children)
    while True:
        # find best child
        children.sort(key = lambda n: n.f) # sort by f value
        bestChild = children[0]
        if bestChild.f > fmax:
            #print(1)
            return ("failure",bestChild.f)
        # next lowest f value
        alternativef = children[1].f if len(children) > 1 else float('inf')
        # expand best child, reassign its f value to be returned value
        result,bestChild.f = aStarSearchHelper(bestChild, actionsF, takeActionF, goalTestF,
                                        hF, min(fmax,alternativef))
        if result is not "failure":      
            print(result)
            result.insert(0,parentNode.state)
            #print(1)
            return (result, bestChild.f)     

In [56]:
aStarSearch('a',actionsF_simple, takeActionF_simple,
            lambda s: goalTestF_simple(s, 'z'),
            lambda s: h_simple(s, 'z'))

['z']
['k', 'z']
['i', 'k', 'z']
['h', 'i', 'k', 'z']
['c', 'h', 'i', 'k', 'z']


(['a', 'c', 'h', 'i', 'k', 'z'], 5)

Download [A3grader.tar](http://www.cs.colostate.edu/~anderson/cs440/notebooks/A3grader.tar) and extract A3grader.py from it.

In [41]:
startState = [1,2,3, 4,0,5, 6,7,8]

goalState =  [1,2,3, 4,0,5, 6,7,8]
goalState2 = [1,2,3, 4,5,8, 6,0,7]
goalState3 = [1,0,3, 4,5,8, 2,6,7]

In [59]:
aStarSearch([1, 2, 3, 4, 5, 6, 7, 0, 8],actionsF_8p, takeActionF_8p,lambda s: goalTestF_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]),lambda s: h1_8p(s, [0, 2, 3, 1, 4,  6, 7, 5, 8]))


1


([[1, 2, 3, 4, 5, 6, 7, 0, 8]], 0)

In [47]:
iterativeDeepeningSearch([5, 2, 8, 0, 1, 4, 3, 7, 6], [0, 2, 3, 1, 4,  6, 7, 5, 8],actionsF_8p, takeActionF_8p, 10)

28739


'cutoff'

In [32]:
def getIDSData(goal):
    res = iterativeDeepeningSearch(startState, goal,actionsF_8p,takeActionF_8p, 20)
    depth = len(res) - 1 
    nodes = getNodeCount()
    branchf = ebf(nodes, depth)
    
    return depth, nodes, branchf

def getAStarData(goal, hfunc):
    def goalTest(state):
        return state == goal
    
    def takeAction(state, action):
        return takeActionF(state, action), 1
    
    res = aStarSearch(startState,actionsF_8p, takeActionF_8p,lambda s: goalTestF_8p(s, startState),lambda s: h1_8p(s, startState))
    depth = len(res[0]) - 1 
    nodes = getNCount()
    branchf = ebf(nodes, depth)
    
    return depth, nodes, branchf

def runExperiment(goalState1, goalState2, goalState3,h=[h1_8p, h2_8p, h3_8p]):
    mformat = '''
            {}    {}    {} 
Algorithm    Depth  Nodes       EBF              Depth  Nodes       EBF              Depth  Nodes       EBF          
     IDS       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}         
    A*h1       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}
    A*h2       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}
    A*h3       {}      {}       {:.3f}              {}      {}      {:.3f}              {}      {}      {:.3f}   
'''

    # IDS results
    print("Finding IDS for goal 1")
    ids1 = getIDSData(goalState)
    print("Finding IDS for goal 2")
    ids2 = getIDSData(goalState2)
    print("Finding IDS for goal 3")
    ids3 = getIDSData(goalState3)
    

    # H1 results
    print("Finding aStar for goal 1, h1")
    aStar1 = getAStarData(goalState, h1_8p)
    print("Finding aStar for goal 2, h1")
    aStar2 = getAStarData(goalState2, h1_8p)
    print("Finding aStar for goal 3, h1")
    aStar3 = getAStarData(goalState3, h1_8p)
      

    # H2 results
    print("Finding aStar for goal 1, h2")
    aStar4 = getAStarData(goalState, h2_8p)
    print("Finding aStar for goal 2, h2")
    aStar5 = getAStarData(goalState2, h2_8p)
    print("Finding aStar for goal 3, h2")
    aStar6 = getAStarData(goalState3, h2_8p)
    
    
    # H3 results
    print("Finding aStar for goal 1, h3")
    aStar7 = getAStarData(goalState, h3_8p)
    print("Finding aStar for goal 2, h3")
    aStar8 = getAStarData(goalState2, h3_8p)
    print("Finding aStar for goal 3, h3")
    aStar9 = getAStarData(goalState3, h3_8p)
   
 
    mformat = mformat.format(goalState,
                             goalState2,
                             goalState3,
                             # IDS
                             ids1[0], ids1[1], ids1[2],
                             ids2[0], ids2[1], ids2[2],
                             ids3[0], ids3[1], ids3[2],
                             # H1
                             aStar1[0], aStar1[1], aStar1[2],
                             aStar2[0], aStar2[1], aStar2[2],
                             aStar3[0], aStar3[1], aStar3[2],
                             # H2
                             aStar4[0], aStar4[1], aStar4[2],
                             aStar5[0], aStar5[1], aStar5[2],
                             aStar6[0], aStar6[1], aStar6[2],
                             # H3
                             aStar7[0], aStar7[1], aStar7[2],
                             aStar8[0], aStar8[1], aStar8[2],
                             aStar9[0], aStar9[1], aStar9[2])

    print(mformat)
    return


In [33]:
runExperiment(goalState, goalState2, goalState3,h=[h1_8p, h2_8p, h3_8p])

Finding IDS for goal 1
Finding IDS for goal 2
Finding IDS for goal 3
Finding aStar for goal 1, h1
Finding aStar for goal 2, h1
Finding aStar for goal 3, h1
Finding aStar for goal 1, h2
Finding aStar for goal 2, h2
Finding aStar for goal 3, h2
Finding aStar for goal 1, h3
Finding aStar for goal 2, h3
Finding aStar for goal 3, h3

            [1, 2, 3, 4, 0, 5, 6, 7, 8]    [1, 2, 3, 4, 5, 8, 6, 0, 7]    [1, 0, 3, 4, 5, 8, 2, 6, 7] 
Algorithm    Depth  Nodes       EBF              Depth  Nodes       EBF              Depth  Nodes       EBF          
     IDS       0      0       0.000              3      43      3.086              11      225850      2.954         
    A*h1       0      38       0.000              0      0      0.000              0      0      0.000
    A*h2       0      0       0.000              0      0      0.000              0      0      0.000
    A*h3       0      0       0.000              0      0      0.000              0      0      0.000   

